In [111]:
import os
from urllib.parse import urlencode
from urllib.parse import quote_plus
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

from sqlalchemy import create_engine
engine = create_engine(os.getenv('PSQL', None) + 'news_item')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Hard Coded URL

In [62]:
tmp = 'https://apis.justwatch.com/content/titles/ko_KR/popular?body=%7B%22content_types%22:[],%22presentation_types%22:[],%22providers%22:[%22nfx%22],%22genres%22:[%22ani%22],%22languages%22:null,%22release_year_from%22:null,%22release_year_until%22:null,%22monetization_types%22:[%22ads%22,%22buy%22,%22flatrate%22,%22free%22,%22rent%22],%22min_price%22:null,%22max_price%22:null,%22scoring_filter_types%22:null,%22age_certifications%22:[],%22timeline_type%22:null,%22page%22:13,%22page_size%22:30%7D'
# r = requests.get(tmp)
# r.json()

# get_url(**kwargs)

In [67]:
# url = 'https://apis.justwatch.com/content/titles/ko_KR/popular?body=%7B%22content_types%22:[],%22presentation_types%22:[],%22providers%22:[%22nfx%22],%22genres%22:[%22ani%22],%22languages%22:None,%22release_year_from%22:None,%22release_year_until%22:None,%22monetization_types%22:[%22ads%22,%22buy%22,%22flatrate%22,%22free%22,%22rent%22],%22min_price%22:None,%22max_price%22:None,%22scoring_filter_types%22:None,%22age_certifications%22:[],%22timeline_type%22:None,%22page%22:13,%22page_size%22:30%7D'

def get_url(**kwargs):
    '''
    page, page_size, providers
    '''
    url_base = 'https://apis.justwatch.com/content/titles/ko_KR/popular?body='
    params = {
        "content_types":[],"presentation_types":[],"providers":["nfx"],"genres":["ani"],"languages":None,"release_year_from":None,
        "release_year_until":None,"monetization_types":["ads","buy","flatrate","free","rent"],"min_price":None,"max_price":None,
        "scoring_filter_types":None,"age_certifications":[],"timeline_type":None,"page":1,"page_size":30
    }
    for k, v in kwargs.items():
        params[k] = v
        print(k, v)
    
    tmp = json.dumps(params)
    tmp = tmp.replace(' ', '')
    params_json = quote_plus(tmp)
    url = url_base + params_json
    return url

# url = get_url(page=2)
# url

In [40]:
r = requests.get(url)
r.json()

{'page': 2,
 'page_size': 30,
 'total_pages': 13,
 'total_results': 368,
 'items': [{'id': 37899,
   'title': '오버로드',
   'full_path': '/kr/TV-프로그램/obeorodeu',
   'full_paths': {'SHOW_DETAIL_OVERVIEW': '/kr/TV-프로그램/obeorodeu'},
   'poster': '/poster/119921016/{profile}',
   'short_description': '가상 세계에서 현실처럼 플레이하는 체감형 게임, 통칭 DMMO-RPG의 전성기 2138년.\n\n2126년에 발매된 《위그드라실》은 DMMO-RPG들 중에서도 방대한 맵과 엄청나게 높은 플레이어 자유도로 인해 국내에서 폭발적인 인기를 얻었고 DMMO-RPG의 붐을 일으킨 장본인이다.  그러나 12년이 지난 지금 조용히 서비스를 종료할 예정이었다.\n\n위그드라실의 최강 마법사, 모몬가는 쓸쓸히 나자릭 지하대분묘에서 종료시간을 기다리고 있었다.  그러나 종료시간을 넘어도 로그아웃되지 않고, NPC들도 갑자기 의지를 가지기 시작하는데.\n\n그의 앞에는 이제껏 본 적 없었던 이세계가 펼쳐지게 된다…!',
   'original_release_year': 2015,
   'object_type': 'show',
   'original_title': 'オーバーロード',
   'original_language': 'ja',
   'max_season_number': 3},
  {'id': 43237,
   'title': '사쿠라코 씨의 발밑에는 시체가 묻혀 있다',
   'full_path': '/kr/TV-프로그램/ying-zi-sannozu-xia-nihasi-ti-gamai-matsuteiru',
   'full_paths': {'SHOW_DETAIL_OVERVIEW': '/kr/TV-프로그램/ying-zi-sannozu-xia-nihasi-

# get_json(url)

In [46]:
def get_json(url):
    r = requests.get(url)
    return r.json()

# Providers

In [60]:
'''
nfx: netflix
nvs: naver store
ply: google playstore
poq: pooq
wac: watcha
'''
providers = ['nfx','nvs','ply','poq','wac']

# get_total_results(provider)

In [71]:
def get_total_results(provider):
    url_init = get_url(providers=[provider])
    res_init = get_json(url_init)
    total_results = int(res_init.get('total_results', '0'))
    return total_results

# total_results = get_total_results('nvs')

providers ['nvs']


# get_df(total_results, provider)

In [74]:
def get_df(total_results, provider):
    url_main = get_url(page_size=total_results, providers=[provider])
    res_main = get_json(url_main)

    df = res_main['items']
    df = pd.DataFrame(df)
    df['provider'] = provider
    return df

# get_animations_from_justwatch(providers)

In [86]:
def get_animations_from_justwatch(providers):
    df = pd.DataFrame()
    for provider in providers:
        total_results = get_total_results(provider)
        tmp = get_df(total_results, provider)
        df = df.append(tmp)

    print(df.shape)
    df.head()

# ani = get_animations_from_justwatch(providers)
# ani.to_csv('animations.tsv', sep='\t', index=False)

# Load ani

In [87]:
ani = pd.read_csv('animations.tsv', sep='\t')
ani.head()

,age_certification,cinema_release_date,cinema_release_week,full_path,full_paths,id,is_nationwide_cinema_release_date,localized_release_date,max_season_number,object_type,original_language,original_release_year,original_title,poster,provider,runtime,short_description,title
0,NaN,2018-07-18,2018_29,/kr/영화/inkeuredeobeul-2,{'MOVIE_DETAIL_OVERVIEW': '/kr/영화/inkeuredeobe...,399,True,2018-07-18,NaN,movie,en,2018,Incredibles 2,/poster/68411762/{profile},nfx,118.0,"세상은 바뀌었고, 우리는 여기에 적응해야 한다고 가족을 설득했던 일라스티걸(홀리 헌...",인크레더블 2
1,NaN,NaN,NaN,/kr/TV-프로그램/voltron-legendary-defender,{'SHOW_DETAIL_OVERVIEW': '/kr/TV-프로그램/voltron-...,42137,NaN,NaN,8.0,show,en,2016,Voltron: Legendary Defender,/poster/65902170/{profile},nfx,NaN,Five unlikely teenage heroes and their flying ...,Voltron: Legendary Defender
2,NaN,NaN,NaN,/kr/TV-프로그램/gengan-asyura,{'SHOW_DETAIL_OVERVIEW': '/kr/TV-프로그램/gengan-a...,106015,NaN,NaN,1.0,show,ja,2019,ケンガンアシュラ,/poster/141461872/{profile},nfx,NaN,기업 간의 분쟁을 맨몸의 승부로 해결한다. 그 비밀스러운 세계로 들어간 도키타 오마...,겐간 아슈라
3,7,NaN,NaN,/kr/TV-프로그램/peppa-pig,{'SHOW_DETAIL_OVERVIEW': '/kr/TV-프로그램/peppa-pig'},22265,NaN,NaN,6.0,show,en,2004,Peppa Pig,/poster/11594230/{profile},nfx,NaN,Peppa Pig is an energetic piggy who lives with...,Peppa Pig
4,NaN,NaN,NaN,/kr/TV-프로그램/teen-titans-go,{'SHOW_DETAIL_OVERVIEW': '/kr/TV-프로그램/teen-tit...,35726,NaN,NaN,5.0,show,en,2013,Teen Titans Go!,/poster/10091861/{profile},nfx,NaN,10대 히어로들의 평범하지만 평범하지 않은 일상을 그린 틴 타이탄의 외전격인 애니메이션,틴 타이탄 GO!


In [88]:
ani.shape

(2691, 18)

In [89]:
ani['id'].nunique()

1241

데이터 길이는 2691인데 유니크 id는 1241개. 플랫폼별로 중복된 애니가 있기 때문.

In [90]:
ani.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2691 entries, 0 to 2690
Data columns (total 18 columns):
age_certification                    43 non-null object
cinema_release_date                  613 non-null object
cinema_release_week                  613 non-null object
full_path                            2691 non-null object
full_paths                           2691 non-null object
id                                   2691 non-null int64
is_nationwide_cinema_release_date    374 non-null object
localized_release_date               613 non-null object
max_season_number                    523 non-null float64
object_type                          2691 non-null object
original_language                    2675 non-null object
original_release_year                2691 non-null int64
original_title                       2691 non-null object
poster                               2683 non-null object
provider                             2691 non-null object
runtime                        

In [101]:
na_count = ani.isna().sum().to_frame()
na_count.reset_index(inplace=True)
na_count.columns = ['column', 'na_count']
na_count.sort_values('na_count', ascending=True)

,column,na_count
17,title,0
3,full_path,0
4,full_paths,0
5,id,0
9,object_type,0
11,original_release_year,0
12,original_title,0
14,provider,0
13,poster,8
10,original_language,16


2691/2691: full_path, full_paths, object_type, original_release_year, original_title, provider,   
2000 이상: 

In [106]:
uni = ani.groupby('id').agg({'original_language':'first'}); uni.head()

,original_language
id,
355,en
391,en
399,en
704,ja
1832,en


In [107]:
uni['original_language'].value_counts()

en    572
ja    437
ko     54
fr     36
zh     26
de     23
es     21
ru     17
sv      6
da      6
no      5
it      5
pt      5
fi      4
cs      3
th      2
ms      1
pl      1
gl      1
hy      1
fa      1
nl      1
sr      1
uk      1
Name: original_language, dtype: int64

In [122]:
uni['original_language'].value_counts().sum()

1230

In [123]:
437/1230 * 100

35.52845528455285

In [121]:
ani.groupby(['provider', 'original_language']).agg({'id':'count'})
lang = ani.pivot_table(index='provider', columns='original_language', values='id', aggfunc='count')
lang.fillna(0, inplace=True)
lang['total'] = lang.sum(axis=1)
lang['en_rate'] = lang['en'] / lang['total'] * 100
lang['jp_rate'] = lang['ja'] / lang['total'] * 100
lang['ko_rate'] = lang['ko'] / lang['total'] * 100
lang['zh_rate'] = lang['zh'] / lang['total'] * 100

lang

original_language,cs,da,de,en,es,fa,fi,fr,gl,hy,it,ja,ko,ms,nl,no,pl,pt,ru,sr,sv,th,uk,zh,total,en_rate,jp_rate,ko_rate,zh_rate
provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
nfx,0.0,1.0,7.0,204.0,3.0,0.0,0.0,11.0,0.0,1.0,2.0,120.0,10.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0,366.0,55.737705,32.786885,2.732240,0.273224
nvs,3.0,5.0,21.0,412.0,17.0,1.0,4.0,24.0,1.0,0.0,3.0,281.0,43.0,1.0,1.0,4.0,1.0,2.0,13.0,0.0,5.0,2.0,1.0,24.0,869.0,47.410817,32.336018,4.948216,2.761795
ply,2.0,2.0,14.0,280.0,13.0,0.0,3.0,13.0,1.0,0.0,1.0,102.0,29.0,1.0,0.0,3.0,1.0,4.0,8.0,1.0,3.0,0.0,1.0,17.0,499.0,56.112224,20.440882,5.811623,3.406814
poq,3.0,3.0,15.0,235.0,13.0,0.0,3.0,20.0,0.0,0.0,1.0,126.0,34.0,1.0,0.0,2.0,1.0,2.0,10.0,1.0,2.0,0.0,1.0,12.0,485.0,48.453608,25.979381,7.010309,2.474227
wac,2.0,3.0,10.0,128.0,7.0,0.0,2.0,16.0,1.0,0.0,1.0,219.0,28.0,1.0,0.0,4.0,1.0,1.0,7.0,0.0,4.0,0.0,1.0,20.0,456.0,28.070175,48.026316,6.140351,4.385965
